# Main Notebook for Automated Multisubject Processing

This is the main overall script for performing technical validation and basic classification.
First the meta data should be loaded and altered based on which subjects and sessions are being used. Also included is the scipt used for converting the EEG data files into BIDS format.



### For Technical Validation:

*Multisensory_EEG_DataSet Save:* This script can be used for saving processed ICA data (with the prior preprocessing steps applied), into .fif files for each subject.


*epoch-save:* This script loads the .fif files and converts them into epochs, and then saves them into folders based on the conditions (multiple subjects in each folder)

*Time Frequency Representation:*  This script computes the intertrial coherence, and also the PSD averaged across trials and subjects for each condition

*ERP:* This script is used to compute the average ERP across subjects for particular tasks and modalities.

### For Classification:

Currently implemented is logistic regression. These scripts can be taken and expanded to implement more advanced classification pipelines, with different feature extraction techniques.

*Basic_Task_Classification:* This script can be used for classifying between perception or imagination tasks for each subject, for each sensory modality


*Classification_Sensory_Modalities:* This script can be used for classifying between the three sensory modalities (pictorial, orthographic or audio) for each subject, and within each task (imagination vs. perception)

*Classification_Semantic:*  This script can be used for classifying between the three semantic categories (penguin, guitar or flower) for each subject, and within each task (imagination vs. perception), and within the three sensory modalities.


In [1]:
import papermill as pm
import pandas as pd
import numpy as np

### Load up the meta data
This includes information on the subject session, cap size, bad channels and any other relevant notes

In [2]:
# load up the parameter csv
param_data = pd.read_csv("X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\meta_extended.csv") 

In [3]:
# reformat the dataframe, reset index to make iteratble
param_data.reset_index()

,index,sub,sess,cap,bad_channels,notes
0,0,3,3,L,"TPP9h, CCP1h",NaN
1,1,8,3,M,CCP2h,NaN
2,2,10,1,M,"CP3, FCC3h, CPP3h",NaN
3,3,11,1,M,"M1, CCP2h",good impedances
4,4,12,1,L,"FC1, C3, Cz, CP1, P3, FCz, C1, CP3, P5, FCC3h,...",NaN
5,5,12,2,L,CCP1h,NaN
6,6,13,1,L,"CCP1h, POO10h",NaN
7,7,14,1,L,NaN,NaN
8,8,14,2,L,NaN,NaN
9,9,15,1,L,NaN,NaN


In [3]:
# drop certain participants based on index
labels = []

param_data = param_data.drop(labels=labels, axis=0)

In [ ]:
param_data

# Code used for converting the files to BIDs format

In [ ]:
for index, row in param_data.iterrows():
    print("For subject ", str(row["sub"]), " and session ", str(row["sess"]), " converting data to bids format")
    pm.execute_notebook(
    'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\convert_bids.ipynb',
   'bids_output.ipynb',
    parameters=dict(ppt_num=str(row["sub"]), session=str(row["sess"]),
    overwrite=True)
    )

## Below is for saving processed ICA data
This runs the *Multisensory_EEG_Preprocess_Save.ipynb* script on all subjects and sessions listed in the meta-extended file.

In [ ]:
# Run each row (assign parameters from file to dictionary)
for index, row in param_data.iterrows():
    print(row['bad_channels'])
    if pd.isna(row['bad_channels']):
        print("there are no bad channels")
        bad_chans = []
        # should replace it with an empty list
    else:
        bad_chans = row['bad_channels'].split(",")
        bad_chans = [s.strip() for s in bad_chans] # get rid of the extra white space that appears
    print(bad_chans)
    print("For subject ", str(row["sub"]), " and session ", str(row["sess"]), " running preprocessing")
    pm.execute_notebook(
    # process bad channel into usable format
    'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\preprocess_automate_chans.ipynb',
   'ica_output.ipynb',
    parameters=dict(ppt_num=str(row["sub"]), session=str(row["sess"]), cap_size = str(row["cap"]), bad_chans=bad_chans)
    )

## Below is for epoching data, and saving it for each condition separately
This uses the *epoch-save.ipynb* script

In [ ]:
# Run each row (assign parameters from file to dictionary)


for index, row in param_data.iterrows():
    print("For subject ", str(row["sub"]), " and session ", str(row["sess"]), " running epoch-saving ready for trf")
    pm.execute_notebook(
    'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\epoch-save.ipynb',
   'epoch_output.ipynb',
    parameters=dict(ppt_num=str(row["sub"]), session=str(row["sess"]), cap_size = str(row["cap"]),
    overwrite=True)
    )

## Computing the time frequency representation analysis
The intertrial coherence and power spectral density plots are computed on the combination of all subjects, but for each condition separately (though ignoring the semantic categories here). Including:
-  imagination orthographic
-  imagination audio
-  imagination pictorial
-  perception orthographic
-  perception audio
-  perception pictorial

The script used is *Time_Frequency_Representation.ipynb*. Output should be the plot images, saved with the title of the condition, and also files with the actual power and ITC numerical values.

In [ ]:
pm.execute_notebook(
   'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\Time_Frequency_Representation.ipynb',
   'tfr_output.ipynb',
   parameters=dict(alpha=0.6, ratio=0.1)
)

## Average ERPs

In [ ]:
pm.execute_notebook(
   'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\ERP.ipynb',
   'erp_output.ipynb',
   parameters=dict(alpha=0.6, ratio=0.1)
)


# Classification

## Classification between Imagination and Perception
Set the task choice (pictorial, orthographic or audio) below, and the number of iterations to average over for classification.

In [ ]:
task = 'pictorial'
iterations = 50

for index, row in param_data.iterrows():
    print("For subject ", str(row["sub"]), " and session ", str(row["sess"]), ' running classification between imagination and perception for ' + task 
          + ' task. This is over '+str(iterations)+' iterations')
    pm.execute_notebook(
    'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\Basic_Task_Classification.ipynb',
   'classification.ipynb',
    parameters=dict(subject=str(row["sub"]), session=str(row["sess"]), task=task, iterations = iterations),
    overwrite=True)
    
    

Black is not installed, parameters wont be formatted


For subject  3  and session  3  running classification between imagination and perception for pictorial task. This is over 50 iterations


Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

## Classification between Sensory Modalities
Classify whether the task is orthographic, audio or pictorial. Parameters to set is whether the overall task is imagination or perception, and the amount of iterations to average over. Results are stored in a .csv file.


In [ ]:
task = 'perception'
iterations = 50

for index, row in param_data.iterrows():
    print("For subject ", str(row["sub"]), " and session ", str(row["sess"]), " running classification between the three sensory modalities")
    pm.execute_notebook(
    'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\Classification_Sensory_Modalities.ipynb',
   'classifications.ipynb',
    parameters=dict(subject=str(row["sub"]), session=str(row["sess"]), task= task, iterations = iterations,
    overwrite=True)
    )

Black is not installed, parameters wont be formatted


For subject  3  and session  3  running classification between the three sensory modalities


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

### Classification between Semantic categories


In [ ]:
modality = 'pictorial'
task = 'perception'
iterations = 50

for index, row in param_data.iterrows():
    print("For subject ", str(row["sub"]), " and session ", str(row["sess"]), " running classification between the three semantic modalities")
    pm.execute_notebook(
    'X:\\CompSci\\ResearchProjects\\EJONeill\\Neuroimaging\\multisensoryeeg\\openNeuro\\code\\Classification_Semantic.ipynb',
   'classification_pictorial.ipynb',
    parameters=dict(subject=str(row["sub"]), session=str(row["sess"]), task= task, modality =modality, iterations = iterations,
    overwrite=True) 
    )